<a href="https://colab.research.google.com/github/Midas0901/Poom/blob/main/ToS_Analyzer_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELL 1: Install everything (run once)
!pip install -q transformers datasets accelerate peft bitsandbytes trl huggingface_hub sentencepiece tqdm

In [ ]:
# CELL 2: Login to Hugging Face (required for pushing model at the end)
from huggingface_hub import login
login()   # ← paste your write token when prompted

In [ ]:
# CELL 3-5 ใหม่: ใช้ MeeraR/legal-qa-dataset (~2,375 ตัวอย่าง Legal/Privacy Q&A) — แก้ ValueError

from datasets import load_dataset
import pandas as pd

print("กำลังโหลดชุดข้อมูล MeeraR/legal-qa-dataset (2,375+ ตัวอย่างจาก Legal/Privacy Policy จริง)...")
dataset = load_dataset("MeeraR/legal-qa-dataset")  # ← ชื่อถูกต้อง!

# เข้าถึงข้อมูลจาก 'train' split โดยตรง (ไม่มี 'file' column — ใช้ item["question"] + item["answer"])
all_data = []
for item in dataset["train"]:  # loop ผ่าน train split (2,375 ตัวอย่าง)
    all_data.append({
        "question": item["question"],
        "answer": item["answer"],
        "source": "Legal Privacy Policy"  # Default source
    })

final_df = pd.DataFrame(all_data)
print(f"โหลดสำเร็จ! ได้ {len(final_df)} ตัวอย่างจริงจาก Legal Q&A")

# เตรียมสำหรับโมเดล (text = question, label = answer)
final_df["text"] = final_df["question"]
final_df["label"] = final_df["answer"]

print(f"Final training size: {len(final_df)} examples (พร้อมฝึกโมเดล!)")
print("\nตัวอย่างแรก:")
print("คำถาม:", final_df.iloc[0]["text"])
print("คำตอบ:", final_df.iloc[0]["label"])

กำลังโหลดชุดข้อมูล MeeraR/legal-qa-dataset (2,375+ ตัวอย่างจาก Legal/Privacy Policy จริง)...


(…)_legal_qa_dataset_unique_questions.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2375 [00:00<?, ? examples/s]

โหลดสำเร็จ! ได้ 2375 ตัวอย่างจริงจาก Legal Q&A
Final training size: 2375 examples (พร้อมฝึกโมเดล!)

ตัวอย่างแรก:
คำถาม: Explain the rights granted by Section 12 of the Indian Penal Code.
คำตอบ: This section ensures legal aid for individuals against sexual harassment.


In [ ]:
# CELL ใหม่: ใช้แค่ 1,200 ตัวอย่าง (สุ่มเลือก + ทำซ้ำได้ทุกครั้ง)

from datasets import Dataset

def make_chat(row):
    return {
        "messages": [
            {"role": "system", "content": "คุณคือผู้เชี่ยวชาญด้าน Privacy Policy และ Terms of Service ตอบให้ชัดเจน อ่านง่าย และอ้างอิงแหล่งที่มาเสมอ ไม่ใช่คำแนะนำทางกฎหมาย"},
            {"role": "user", "content": row["text"]},
            {"role": "assistant", "content": row["label"]}
        ]
    }

# สร้าง chat list ทั้งหมดก่อน
chat_list = [make_chat(row) for _, row in final_df.iterrows()]

# เลือกแค่ 1,200 ตัวอย่างแบบสุ่ม (seed คงที่ → ผลเหมือนกันทุกทุกครั้ง)
import random
random.seed(42)  # สำคัญ! เพื่อให้สุ่มซ้ำได้
selected_examples = random.sample(chat_list, 1200)

# แปลงเป็น Dataset
full_dataset = Dataset.from_list(selected_examples)
full_dataset = full_dataset.train_test_split(test_size=0.1, seed=42)  # 90% train, 10% test

print(f"พร้อมฝึก! ใช้แค่ 1,200 ตัวอย่างตามที่ต้องการ")
print(f"Train: {len(full_dataset['train'])}, Test: {len(full_dataset['test'])}")

พร้อมฝึก! ใช้แค่ 1,200 ตัวอย่างตามที่ต้องการ
Train: 1080, Test: 120


In [ ]:
# CELL 6: เวอร์ชันสุดท้าย 100% ทำงาน – โหลดเสร็จใน 30-60 วินาทีจริง ๆ

!pip install -q -U transformers accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"   # ชื่อที่ถูกต้องที่สุด

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("กำลังโหลด tokenizer... (10 วินาที)")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("กำลังโหลดโมเดล... (30-60 วินาที — เร็วที่สุดแล้วใน Colab ฟรี)")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16,        # ใส่แค่อันเดียวพอ
    low_cpu_mem_usage=True
)

print("โหลดเสร็จสมบูรณ์! VRAM:", round(torch.cuda.memory_allocated()/1e9, 2), "GB")
print("ไปต่อ CELL ถัดไป (ฝึกโมเดล) ได้เลยครับ")

กำลังโหลด tokenizer... (10 วินาที)


`torch_dtype` is deprecated! Use `dtype` instead!


กำลังโหลดโมเดล... (30-60 วินาที — เร็วที่สุดแล้วใน Colab ฟรี)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# CELL 7: Fine-tune with LoRA (1–3 hours)

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import transformers

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules="all-linear",
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=full_dataset["train"],
    eval_dataset=full_dataset["test"],
    dataset_text_field="messages",
    tokenizer=tokenizer,
    max_seq_length=2048,
    formatting_func=lambda examples: [tokenizer.apply_chat_template(ex["messages"], tokenize=False) for ex in examples],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        output_dir="raft-tos-analyzer",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        evaluation_strategy="epoch",
        report_to="none",
    ),
)

print("Starting training...")
trainer.train()

In [ ]:
# CELL 8: Merge and save final model

model.save_pretrained("raft-tos-analyzer-final")
tokenizer.save_pretrained("raft-tos-analyzer-final")

# Merge LoRA weights
!pip install -q peft
from peft import AutoPeftModelForCausalLM

merged_model = AutoPeftModelForCausalLM.from_pretrained(
    "raft-tos-analyzer-final",
    device_map="auto",
    torch_dtype=torch.float16
)
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained("raft-tos-analyzer-merged")
tokenizer.save_pretrained("raft-tos-analyzer-merged")

In [ ]:
# CELL 9: Push your final model to Hugging Face

new_model_name = "yourusername/raft-tos-analyzer-v1"   # ← CHANGE "yourusername" !

merged_model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)

print("FINISHED!")
print(f"Your model is now live at: https://huggingface.co/{new_model_name}")

In [ ]:
# CELL 10: Quick test

from transformers import pipeline
pipe = pipeline("text-generation", model=new_model_name, device_map="auto")

test = pipe("Does this app sell my personal data?", max_new_tokens=200)
print(test[0]["generated_text"])